<a href="https://colab.research.google.com/github/EagleVector/tfx_taxi/blob/main/tfx_components.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building Pipeline Components using TensorFlow Extend(TFX)

In [30]:
cd /content/drive/MyDrive/tfx

/content/drive/MyDrive/tfx


In [2]:
cd /content/drive/MyDrive/tfx/data

/content/drive/MyDrive/tfx/data


## Chicago Taxi Dataset

In [3]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head()

,pickup_community_area,fare,trip_start_month,trip_start_hour,trip_start_day,trip_start_timestamp,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_miles,pickup_census_tract,dropoff_census_tract,payment_type,company,trip_seconds,dropoff_community_area,tips
0,60,27.05,10,2,3,1380593700,41.836150,-87.648788,NaN,NaN,12.6,NaN,NaN,Cash,Taxi Affiliation Services,1380.0,NaN,0.0
1,10,5.85,10,1,2,1382319000,41.985015,-87.804532,NaN,NaN,0.0,NaN,NaN,Cash,Taxi Affiliation Services,180.0,NaN,0.0
2,14,16.65,5,7,5,1369897200,41.968069,-87.721559,NaN,NaN,0.0,NaN,NaN,Cash,Dispatch Taxi Affiliation,1080.0,NaN,0.0
3,13,16.45,11,12,3,1446554700,41.983636,-87.723583,NaN,NaN,6.9,NaN,NaN,Cash,NaN,780.0,NaN,0.0
4,16,32.05,12,1,1,1417916700,41.953582,-87.723452,NaN,NaN,15.4,NaN,NaN,Cash,NaN,1200.0,NaN,0.0


## Installing dependencies

In [5]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 4.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


In [6]:
!pip install tfx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.6/25.6 MB 42.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 47.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

# Data Ingestion

In [31]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

The tfx.orchestration.experimental.interactive.notebook_extensions.skip extension is already loaded. To reload it, use:
  %reload_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip


In [32]:
!ls

data  diabetes.csv  parquet_dir  tf_record_files  Untitled0.ipynb


## Setting up the configuration for Data Ingestion Phase.

In [33]:
class Config():
  
  def __init__(self):
    self.data_dir = os.path.join("data")
    self.csv_file_name = "data.csv"
    self.csv_file_path = os.path.join(self.data_dir, self.csv_file_name)
    self.pipeline_name = 'aiops_pipeline'
    self.pipeline_root = os.path.join("aiops_pipeline_artifact")
    self.meta_data_path = os.path.join("aiops_pipeline_metadata", self.pipeline_name, "metadata.db")


In [34]:
config = Config()

## How to run the pipeline built using TFX



1.   Interactive Context
2.   Local DAG Runner
3.   Beam DAG Runner
4.   Airflow
5.   Kubeflow



Interactive jupyter notebook in the experimental phase.

TFX internally supports databases that stores all the relevant metadata during the execution of pipeline.

In [35]:
from tfx.orchestration.metadata import sqlite_metadata_connection_config

In [36]:
interactive_context = InteractiveContext(
    pipeline_name=config.pipeline_name,
    pipeline_root=config.pipeline_root,
    metadata_connection_config=sqlite_metadata_connection_config(config.meta_data_path)
)

In [37]:
!ls

aiops_pipeline_metadata  diabetes.csv  tf_record_files
data			 parquet_dir   Untitled0.ipynb


In [38]:
from tfx.components import CsvExampleGen

## CsvExampleGen is used to convert the csv data into tf reader format

In [39]:
csv_example_gen = CsvExampleGen(input_base=config.data_dir)

## Artifact of the Data Ingestion Phase built using InteractiveContext


In [40]:
interactive_context.run(csv_example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [ ]:
pip install apache-beam[interactive]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.7/186.7 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.0/424.0 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 45.8 MB/s eta 0:00:00
  Created wheel for timeloop: filename=timeloop-1.0.2-py3-none-any.whl size=3720 sha256=66c9f9f640a450a9197c990363142443eafcacd1aa9c4cb7bb57ae9c0740f047
  Stored in directory: /root/.cache/pip/wheels/b4/64/75/079be5e9f033584ae370bbd2de7668cd5b9e14b73661c3595a
Successfully built timeloop
  Attempting uninstall: tornado
    Found existing installation: tornado 6.0.4
    Uninstalling tornado-6.0.4:
      Successfully uninstalled tornado-6.0.4
  Attempting uninstall: ipython
    Found existing installation: ipython 7.9.0
    Uninstall

Getting the list of all the files in a present directory.

In [41]:
os.listdir()

['data',
 '.ipynb_checkpoints',
 'diabetes.csv',
 'tf_record_files',
 'parquet_dir',
 'Untitled0.ipynb',
 'aiops_pipeline_metadata',
 'aiops_pipeline_artifact']

In [ ]:
os.listdir()

['data',
 '.ipynb_checkpoints',
 'aiops_pipeline_metadata',
 'aiops_pipeline_artifact',
 'diabetes.csv',
 'tf_record_files',
 'Untitled0.ipynb']

## CsvExampleGen Insights
1.Example

2.---Features

3.-----Feature

4.--------{column name: value}






In [70]:
csv_file_path = os.path.join(os.getcwd(),"data","data.csv")

In [71]:
os.path.exists(csv_file_path)

True

In [72]:
csv_file_path

'/content/drive/MyDrive/tfx/data/data.csv'

In [75]:
import csv

In [76]:
import tensorflow as tf

In [77]:
tf_record_dir = os.path.join(os.getcwd(), "tf_record_files")

In [78]:
os.makedirs(tf_record_dir, exist_ok=True)

In [79]:
os.listdir()

['data',
 '.ipynb_checkpoints',
 'diabetes.csv',
 'parquet_dir',
 'Untitled0.ipynb',
 'aiops_pipeline_metadata',
 'aiops_pipeline_artifact',
 'tf_record_files']

In [80]:
tf_record_file_name=os.path.join(tf_record_dir, "data.tfrecord")

File to save the data in tf reader format

In [81]:
tf_record_writer = tf.io.TFRecordWriter(tf_record_file_name)

## An illustration to show the conversion of csv file to tf reader format.

In [ ]:
import pandas as pd
df = pd.read_csv(csv_file_path)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
def _bytes_features(value):
  value = value.encode()
  return tf.train.BytesList(value=[value])

In [ ]:
def _int64_features(value):
  return tf.train.Int64List(value=[value])

In [ ]:
def _float_features(value):
  return tf.train.FloatList(value=[value])

In [ ]:
_bytes_features(value = 'shubham')

value: "shubham"

In [ ]:
_int64_features(value=56)

value: 56

In [ ]:
_float_features(value=78.99)

value: 78.98999786376953

In [ ]:
tf.train.Feature(bytes_list=_bytes_features("cherry"))

bytes_list {
  value: "cherry"
}

In [ ]:
tf.train.Example(features = tf.train.Features(feature={"name":tf.train.Feature(bytes_list=_bytes_features("cherry")),
                                                       "age": tf.train.Feature(int64_list=_int64_features(26)),
                                                       "weight": tf.train.Feature(float_list=_float_features(71.80))}))

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 26
      }
    }
  }
  feature {
    key: "name"
    value {
      bytes_list {
        value: "cherry"
      }
    }
  }
  feature {
    key: "weight"
    value {
      float_list {
        value: 71.80000305175781
      }
    }
  }
}

This represents the first row in the tf record file.

In [ ]:
example = tf.train.Example(features = tf.train.Features(feature={"name":tf.train.Feature(bytes_list=_bytes_features("cherry")),
                                                       "age": tf.train.Feature(int64_list=_int64_features(26)),
                                                       "weight": tf.train.Feature(float_list=_float_features(71.80))}))

## serialized data

In [ ]:
example.SerializeToString()

b'\n6\n\x12\n\x06weight\x12\x08\x12\x06\n\x04\x9a\x99\x8fB\n\x12\n\x04name\x12\n\n\x08\n\x06cherry\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01\x1a'

This is the internal representation of tf reader file.

## Slightly modification->Automation

In [82]:
def _bytes_feature(value):
  value = value.encode()
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

In [83]:
def _int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [84]:
def _float_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

## Converted few features into tf reader format.

In [85]:
with open(csv_file_path) as csv_file:
  reader = csv.DictReader(csv_file, delimiter=",", quotechar='"')
  for row in reader:
    feature={
        'pickup_community_area':_int64_feature(int(row['pickup_community_area'])),
        'trip_start_month':_int64_feature(int(row['trip_start_month'])),
        'trip_start_hour':_int64_feature(int(row['trip_start_hour'])),
        'company':_bytes_feature(str(row['company'])),
        'fare':_float_feature(float(row['fare']))
    }
    features=tf.train.Features(feature=feature)
    example=tf.train.Example(features=features)
    tf_record_writer.write(example.SerializeToString())

tf_record_writer.close()

In [86]:
feature

{'pickup_community_area': int64_list {
   value: 61
 }, 'trip_start_month': int64_list {
   value: 4
 }, 'trip_start_hour': int64_list {
   value: 14
 }, 'company': bytes_list {
   value: "Taxi Affiliation Services"
 }, 'fare': float_list {
   value: 5.650000095367432
 }}

In [87]:
row

OrderedDict([('pickup_community_area', '61'),
             ('fare', '5.65'),
             ('trip_start_month', '4'),
             ('trip_start_hour', '14'),
             ('trip_start_day', '6'),
             ('trip_start_timestamp', '1398436200'),
             ('pickup_latitude', '41.80901825'),
             ('pickup_longitude', '-87.659166599'),
             ('dropoff_latitude', '41.817366208'),
             ('dropoff_longitude', '-87.698860797'),
             ('trip_miles', '1.0'),
             ('pickup_census_tract', ''),
             ('dropoff_census_tract', ''),
             ('payment_type', 'Cash'),
             ('company', 'Taxi Affiliation Services'),
             ('trip_seconds', '300'),
             ('dropoff_community_area', '58'),
             ('tips', '0.0')])

In [88]:
row['company']

'Taxi Affiliation Services'

### Creating a parquet file from the csv file using pandas.

In [42]:
csv_file_path = os.path.join(os.getcwd(), "data", "data.csv")
csv_file_path

'/content/drive/MyDrive/tfx/data/data.csv'

In [43]:
parquet_file_dir = os.path.join(os.getcwd(), "parquet_dir")

In [44]:
os.getcwd()

'/content/drive/MyDrive/tfx'

In [45]:
os.makedirs(parquet_file_dir, exist_ok=True)

In [46]:
parquet_file_name = "data.parquet"

In [47]:
parquet_file_path = os.path.join(parquet_file_dir, parquet_file_name)

In [48]:
parquet_file_path

'/content/drive/MyDrive/tfx/parquet_dir/data.parquet'

In [50]:
import pandas as pd
pd.read_csv(csv_file_path).to_parquet(parquet_file_path)

## Ingesting parquet data

In [51]:
from tfx.components import FileBasedExampleGen

In [52]:
from tfx.components.base import executor_spec

In [53]:
from tfx.components.example_gen.custom_executors import parquet_executor

In [54]:
parquet_dataset_dir = os.path.join(os.getcwd(), "parquet_dir")

In [55]:
file_based_example_gen = FileBasedExampleGen(input_base=parquet_dataset_dir, custom_executor_spec=executor_spec.ExecutorClassSpec(parquet_executor.Executor))

In [56]:
interactive_context.run(file_based_example_gen)

ExecutionResult(
    component_id: FileBasedExampleGen
    execution_id: 2
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=FileBasedExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

The same step will be followed in case we have avro file. There we will have to import avro_executor.

# Tensorflow Data Validation

In [89]:
import tensorflow_data_validation as tfdv

In [93]:
tf_record_file = os.path.join(os.getcwd(), "tf_record_files","data.tfrecord")

In [94]:
tf_record_file

'/content/drive/MyDrive/tfx/tf_record_files/data.tfrecord'

In [95]:
os.path.exists(tf_record_file)

True

In [96]:
tfdv.generate_statistics_from_tfrecord(tf_record_file)

From /usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/statistics_io_impl.py:91: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


datasets {
  num_examples: 15000
  features {
    type: STRING
    string_stats {
      common_stats {
        num_non_missing: 15000
        min_num_values: 1
        max_num_values: 1
        avg_num_values: 1.0
        num_values_histogram {
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1500.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1500.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1500.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1500.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1500.0
          }
          buckets {
            low_value: 1.0
            high_value: 1.0
            sample_count: 1500.0
          }
          buckets {
            low

Difficult to interpret this form of data. We will use visualization tool from tensorflow.

In [98]:
stats = tfdv.generate_statistics_from_tfrecord(tf_record_file)

In [99]:
tfdv.visualize_statistics(stats)

In [100]:
schema = tfdv.infer_schema(statistics=stats)

In [101]:
schema

feature {
  name: "company"
  type: BYTES
  domain: "company"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "fare"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "pickup_community_area"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "trip_start_hour"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "trip_start_month"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
string_domain {
  name: "company"
  value: ""
  value: "0118 - 42111 Godfrey S.Awir"
  value: "0694 - 59280 Chinesco Trans Inc"
  value: "1085 - 72312 N and W Cab Co"
  value: "2092 - 61288 Sbeih company"
  value: "2192 - 73487 Zeymane Co

### Data Validation setup for missing values.

In [102]:
feature_schema = tfdv.get_feature(schema, "fare")

min_fraction gives the statistics of percentage of data abundancy.

In [103]:
feature_schema.presence.min_fraction = 0.95

In [104]:
feature_schema

name: "fare"
type: FLOAT
presence {
  min_fraction: 0.95
  min_count: 1
}
shape {
  dim {
    size: 1
  }
}

This will allow the data pass upto 5% of missing values.

In [105]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'company',STRING,required,,'company'
'fare',FLOAT,optional,,-
'pickup_community_area',INT,required,,-
'trip_start_hour',INT,required,,-
'trip_start_month',INT,required,,-


,Values
Domain,
'company',"'', '0118 - 42111 Godfrey S.Awir', '0694 - 59280 Chinesco Trans Inc', '1085 - 72312 N and W Cab Co', '2092 - 61288 Sbeih company', '2192 - 73487 Zeymane Corp', '2192 - Zeymane Corp', '2733 - 74600 Benny Jona', '2809 - 95474 C & D Cab Co Inc.', '2823 - 73307 Seung Lee', '3011 - 66308 JBL Cab Inc.', '3094 - 24059 G.L.B. Cab Co', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3319 - CD Cab Co', '3385 - 23210 Eman Cab', '3385 - Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - 57856 Ilie Malec', '3897 - Ilie Malec', '4053 - 40193 Adwar H. Nikola', '4053 - Adwar H. Nikola', '4197 - 41842 Royal Star', '4197 - Royal Star', '4615 - 83503 Tyrone Henderson', '4615 - Tyrone Henderson', '4623 - Jay Kim', '5006 - 39261 Salifu Bawa', '5006 - Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - 98755 Mengisti Taxi', '5129 - Mengisti Taxi', '5724 - KYVI Cab Inc', '585 - 88805 Valley Cab Co', '585 - Valley Cab Co', '5864 - 73614 Thomas Owusu', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5874 - Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '5997 - AW Services Inc.', '6057 - 24657 Richard Addo', '6488 - 83287 Zuha Taxi', '6574 - Babylon Express Inc.', '6742 - 83735 Tasha ride inc', '6743 - Luhak Corp', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation'"


In [106]:
tfdv.write_schema_text(schema=schema, output_path="set_schema.txt")

Anomaly detection in the incoming new data.

In [107]:
anomalies = tfdv.validate_statistics(statistics=stats, schema=schema)

Check for anomaly detection.

In [108]:
tfdv.display_anomalies(anomalies)

TFX gives a very clear report on anomaly if it is present.

In [110]:
data_stats = tfdv.generate_statistics_from_csv(os.path.join(os.getcwd(), "data", "data.csv"))

In [111]:
tfdv.display_anomalies(tfdv.validate_statistics(statistics=data_stats, schema = schema))

,Anomaly short description,Anomaly long description
Feature name,,
'company',Multiple errors,"The feature has a shape, but it's not always present (if the feature is nested, then it should always be present at each nested level) or its value lengths vary. The feature was present in fewer examples than expected: minimum fraction = 1.000000, actual = 0.657333"
'dropoff_latitude',New column,New column (column in data but not in schema)
'trip_start_day',New column,New column (column in data but not in schema)
'trip_start_timestamp',New column,New column (column in data but not in schema)
'pickup_longitude',New column,New column (column in data but not in schema)
'dropoff_longitude',New column,New column (column in data but not in schema)
'trip_miles',New column,New column (column in data but not in schema)
'dropoff_census_tract',New column,New column (column in data but not in schema)
'pickup_latitude',New column,New column (column in data but not in schema)


Performing the Data Validation in TFX using StatisticsGen, SchemaGen, ExampleValidator

In [112]:
from tfx.components import StatisticsGen, SchemaGen, ExampleValidator

In [115]:
statistics_gen = StatisticsGen(examples = csv_example_gen.outputs['examples'])

In [114]:
csv_example_gen.outputs['examples']

OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={})

Comparision will be there in data validation from the already generated artifact.

Lets run our Data Validation component using the same runner i.e InteractiveContext.

In [116]:
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 3
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

The input is coming from metadata.db

Statistical visualization of the entire data validation stage.

In [118]:
interactive_context.show(statistics_gen.outputs['statistics'])

From here we can do the entire analysis on data

In [119]:
schema_gen = SchemaGen(statistics = statistics_gen.outputs['statistics'])

In [120]:
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 4
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [121]:
interactive_context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',STRING,required,,'company'
'dropoff_census_tract',INT,required,,-
'dropoff_community_area',INT,required,,-
'dropoff_latitude',FLOAT,required,,-
'dropoff_longitude',FLOAT,required,,-
'fare',FLOAT,required,,-
'payment_type',STRING,required,,'payment_type'
'pickup_census_tract',BYTES,optional,,-
'pickup_community_area',INT,required,,-


,Values
Domain,
'company',"'0118 - 42111 Godfrey S.Awir', '0694 - 59280 Chinesco Trans Inc', '1085 - 72312 N and W Cab Co', '2092 - 61288 Sbeih company', '2192 - 73487 Zeymane Corp', '2192 - Zeymane Corp', '2733 - 74600 Benny Jona', '2809 - 95474 C & D Cab Co Inc.', '2823 - 73307 Seung Lee', '3011 - 66308 JBL Cab Inc.', '3094 - 24059 G.L.B. Cab Co', '3152 - 97284 Crystal Abernathy', '3201 - C&D Cab Co Inc', '3201 - CID Cab Co Inc', '3253 - 91138 Gaither Cab Co.', '3385 - 23210 Eman Cab', '3385 - Eman Cab', '3623 - 72222 Arrington Enterprises', '3897 - 57856 Ilie Malec', '3897 - Ilie Malec', '4053 - Adwar H. Nikola', '4197 - 41842 Royal Star', '4197 - Royal Star', '4615 - 83503 Tyrone Henderson', '4615 - Tyrone Henderson', '4623 - Jay Kim', '5006 - 39261 Salifu Bawa', '5074 - 54002 Ahzmi Inc', '5074 - Ahzmi Inc', '5129 - 87128', '5129 - 98755 Mengisti Taxi', '5129 - Mengisti Taxi', '585 - Valley Cab Co', '5864 - 73614 Thomas Owusu', '5864 - Thomas Owusu', '5874 - 73628 Sergey Cab Corp.', '5874 - Sergey Cab Corp.', '5997 - 65283 AW Services Inc.', '5997 - AW Services Inc.', '6488 - 83287 Zuha Taxi', '6574 - Babylon Express Inc.', '6742 - 83735 Tasha ride inc', 'Blue Ribbon Taxi Association Inc.', 'C & D Cab Co Inc', 'Chicago Elite Cab Corp.', 'Chicago Elite Cab Corp. (Chicago Carriag', 'Chicago Medallion Leasing INC', 'Chicago Medallion Management', 'Choice Taxi Association', 'Dispatch Taxi Affiliation', 'KOAM Taxi Association', 'Northwest Management LLC', 'Taxi Affiliation Services', 'Top Cab Affiliation', '3319 - CD Cab Co', '4053 - 40193 Adwar H. Nikola', '5006 - Salifu Bawa', '5724 - KYVI Cab Inc', '585 - 88805 Valley Cab Co', '6057 - 24657 Richard Addo', '6743 - Luhak Corp'"
'payment_type',"'Cash', 'Credit Card', 'Dispute', 'No Charge', 'Pcard', 'Prcard', 'Unknown'"


ExampleValidator will be used for breaking the pipeline in case of any mis match in data validation phase. Further execution of pipeline will be stopped.

In [122]:
example_val = ExampleValidator(statistics=statistics_gen.outputs['statistics'], schema=schema_gen.outputs['schema'])

In [123]:
interactive_context.run(example_val)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 5
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

REPORT

In [125]:
interactive_context.show(example_val.outputs['anomalies'])

## Data Transformation

In [133]:
from tfx.components import Transform

In [128]:
import pandas as pd
df = pd.read_csv('data/data.csv')
df.head()

,pickup_community_area,fare,trip_start_month,trip_start_hour,trip_start_day,trip_start_timestamp,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_miles,pickup_census_tract,dropoff_census_tract,payment_type,company,trip_seconds,dropoff_community_area,tips
0,60,27.05,10,2,3,1380593700,41.836150,-87.648788,NaN,NaN,12.6,NaN,NaN,Cash,Taxi Affiliation Services,1380.0,NaN,0.0
1,10,5.85,10,1,2,1382319000,41.985015,-87.804532,NaN,NaN,0.0,NaN,NaN,Cash,Taxi Affiliation Services,180.0,NaN,0.0
2,14,16.65,5,7,5,1369897200,41.968069,-87.721559,NaN,NaN,0.0,NaN,NaN,Cash,Dispatch Taxi Affiliation,1080.0,NaN,0.0
3,13,16.45,11,12,3,1446554700,41.983636,-87.723583,NaN,NaN,6.9,NaN,NaN,Cash,NaN,780.0,NaN,0.0
4,16,32.05,12,1,1,1417916700,41.953582,-87.723452,NaN,NaN,15.4,NaN,NaN,Cash,NaN,1200.0,NaN,0.0


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   pickup_community_area   15000 non-null  int64  
 1   fare                    15000 non-null  float64
 2   trip_start_month        15000 non-null  int64  
 3   trip_start_hour         15000 non-null  int64  
 4   trip_start_day          15000 non-null  int64  
 5   trip_start_timestamp    15000 non-null  int64  
 6   pickup_latitude         15000 non-null  float64
 7   pickup_longitude        15000 non-null  float64
 8   dropoff_latitude        14520 non-null  float64
 9   dropoff_longitude       14520 non-null  float64
 10  trip_miles              15000 non-null  float64
 11  pickup_census_tract     0 non-null      float64
 12  dropoff_census_tract    10761 non-null  float64
 13  payment_type            15000 non-null  object 
 14  company                 9860 non-null 

Custom TFX component for data transformation(based on the requirement)

In [131]:
import tensorflow_transform as tft

In [138]:
_DENSE_FLOAT_FEATURE_KEYS = ['trip_miles', 'fare', 'trip_seconds']
_BUCKET_FEATURE_KEYS = ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']
_VOCAB_FEATURE_KEYS = ['payment_type', 'company']
_LABEL_KEY = 'tips'
_FARE_KEY = 'fare'


_VOCAB_SIZE=10000
_OOV_SIZE =10
_BUCKET_SIZE = 10

def _transformed_name(key):
  return key + '_xf'

def _transformed_names(keys):
  return [_transformed_name(key) for key in keys]

def _fill_in_missing(x):
  default_value = '' if x.dtype == tf.string else 0
  return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value
        ))
  
def preprocessing_fn(input):
    output={}
    print(f"Applying standardization on columns: {_DENSE_FLOAT_FEATURE_KEYS}")
    for key in _DENSE_FLOAT_FEATURE_KEYS:
        output[_transformed_name(key)] = tft.scale_to_z_score(
            _fill_in_missing(input[key]))

    print(f"Applying vocabulary lookup on columns: {_VOCAB_FEATURE_KEYS}")
    for key in _VOCAB_FEATURE_KEYS:
        output[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
            _fill_in_missing(input[key]),top_k =_VOCAB_SIZE, 
            oov_buckets = _OOV_SIZE)
    print(f"Applying bucketization on columns: {_BUCKET_FEATURE_KEYS}")
    for key in _BUCKET_FEATURE_KEYS:
        output[_transformed_name(key)] = tft.bucketize(
            _fill_in_missing(input[key]),_BUCKET_SIZE)
        
    taxi_fare = _fill_in_missing(input[_FARE_KEY])
    tips = _fill_in_missing(input[_LABEL_KEY])

    output[_transformed_name(_LABEL_KEY)] = tf.where(
        tf.math.is_nan(taxi_fare),
        tf.cast(tf.zeros_like(taxi_fare), tf.int64),
        tf.cast(
            tf.greater(tips, tf.multiply(taxi_fare, tf.constant(0.10))),
            tf.int64),
        )
    return output

In [145]:
transform = Transform(examples=csv_example_gen.outputs['examples'],
                      schema = schema_gen.outputs['schema'],
                      module_file=os.path.join(os.getcwd(), "feature engineering", "transform.py"))

In [ ]:
interactive_context.run(transform)

Error: Tensor does not have 'indices'

Some version of tensorflow dont support SparseTensor indices